In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import itertools as it
from unidecode import unidecode # For converting lithuanian characters to latin

In [2]:
""""
Issues: 
1- For 7bet, find a method to get api url dynamically. "File" always starts with "GetEvents?"
2- 7bet NHL has 2 odds, while others has 3 odds. So, it's not included
3- Merge dataframes from all, not just from df_topsport
"""

'"\nIssues: \n1- For 7bet, find a method to get api url dynamically. "File" always starts with "GetEvents?"\n2- 7bet NHL has 2 odds, while others has 3 odds. So, it\'s not included\n3- Merge dataframes from all, not just from df_topsport\n'

## Scrape Topsport, 7bet and betsafe

In [3]:
# Set all urls
url_list_topsport = ["https://www.topsport.lt/krepsinis/nba",
                     "https://www.topsport.lt/krepsinis/eurolyga",
                     "https://www.topsport.lt/futbolas/uefa-europos-lyga",
                     "https://www.topsport.lt/uefa-europos-konferenciju-lyga",
                     "https://www.topsport.lt/amerikietiskas-futbolas/jav",
                     "https://www.topsport.lt/ledo-ritulys/nhl",
                     "https://www.topsport.lt/odds/all/10/0/1459" # Smiginis
                     ]

# Set all league names (ending of each url)
league_names = ['NBA', 'Eurolyga', 'UEFA', 'UEFA Konf.', "NFL", "NHL", "Smiginis"]

def scrape_topsport(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    today = datetime.today()

    events = soup.find_all('div', {'class': 'js-prelive-event-row'}) # Find all events

    matches_topsport = []
    for event in events:
        try:
            date = event.find('span', {'class':'prelive-event-date'}).text.lower()
            teams = event.find_all('div', {'class': 'prelive-outcome-buttons-item-title-overflow'})
            odds = event.find_all('span', "prelive-list-league-rate ml-1 h-font-secondary h-fs17 h-fw500")

            # Convert "Today" and "Tomorrow" to actual dates
            if "šiandien" in date:
                date = today.strftime("%Y-%m-%d ") + date.split(" ")[1]
            elif "rytoj" in date:
                tomorrow = today + timedelta(days=1)
                date = tomorrow.strftime("%Y-%m-%d ") + date.split(" ")[1]

            # Ensure we have both teams and their respective odds
            if len(teams) == 2 and len(odds) == 2:
                team1 = unidecode(teams[0].text.strip()) # Convert lt characters to latin, if exists
                team2 = unidecode(teams[1].text.strip())
                odd1 = float(odds[0].text)
                odd2 = float(odds[1].text)
                
                if team1 == "Taip" or team2 == "Taip": # Skip extra bets with "yes" "no" options
                    continue

                matches_topsport.append((date, (team1, team2), (odd1, odd2)))

            # For games with 3 odds like football
            elif len(teams) == 3 and len(odds) == 3:
                team1 = unidecode(teams[0].text.strip()) # Convert lt characters to latin, if exists
                team2 = unidecode(teams[2].text.strip())
                odd1 = float(odds[0].text)
                odd2 = float(odds[1].text)
                odd3 = float(odds[2].text)
                
                if team1 == "Yes" or team2 == "Yes": # Skip extra bets with "yes" "no" options
                    continue

                matches_topsport.append((date, (team1, "Draw" ,team2), (odd1, odd2, odd3)))
        except:
            continue

    return matches_topsport

all_matches = []

for i, league_name in enumerate(league_names):
    matches = scrape_topsport(url_list_topsport[i])
    all_matches.extend([(date, teams, odds, league_name) for date, teams, odds in matches])

# Create a single DataFrame
df_topsport = pd.DataFrame(all_matches, columns=["Date", "Teams", "Odds", "League"])

# df_topsport

In [4]:
# Define leagues. Integers represent (number of odds, increment) for that sport
# Increment is how many bets to skip in api
leagues = {"NBA": (2, "https://sb2frontend-altenar2.biahosted.com/api/widget/GetEvents"),
           "Eurolyga": (2, "https://sb2frontend-altenar2.biahosted.com/api/widget/GetEvents?culture=lt-LT&timezoneOffset=-120&integration=7bet&deviceType=1&numFormat=en-GB&countryCode=LT&eventCount=0&sportId=0&champIds=2995"),
           "UEFA": (3, "https://sb2frontend-altenar2.biahosted.com/api/widget/GetEvents?culture=lt-LT&timezoneOffset=-120&integration=7bet&deviceType=1&numFormat=en-GB&countryCode=LT&eventCount=0&sportId=0&champIds=16809"),
           "UEFA Konf.": (3, "https://sb2frontend-altenar2.biahosted.com/api/widget/GetEvents?culture=lt-LT&timezoneOffset=-120&integration=7bet&deviceType=1&numFormat=en-GB&countryCode=LT&eventCount=0&sportId=0&champIds=31608"),
           "NFL": (2, "https://sb2frontend-altenar2.biahosted.com/api/widget/GetEvents?culture=lt-LT&timezoneOffset=-120&integration=7bet&deviceType=1&numFormat=en-GB&countryCode=LT&eventCount=0&sportId=0&champIds=3281"),
           "Smiginis": (2, "https://sb2frontend-altenar2.biahosted.com/api/widget/GetEvents?culture=lt-LT&timezoneOffset=-120&integration=7bet&deviceType=1&numFormat=en-GB&countryCode=LT&eventCount=0&sportId=0&champIds=3288"),
           }

params = {
    "culture": "lt-LT",
    "timezoneOffset": "-120",
    "integration": "7bet",
    "deviceType": "1",
    "numFormat": "en-GB",
    "countryCode": "LT",
    "eventCount": "0",
    "sportId": "0",
    "champIds": "2980"
}

# Add headers copied from browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Referer": "https://7bet.lt/", 
    "Origin": "https://7bet.lt",
    "Accept": "application/json, text/plain, */*"
}

def remove_duplicates(events, only_duplicates = False):
    duplicates = []
    for idx, event in enumerate(events):
        # Skip first event
        if idx == 0: 
            continue
        
        if only_duplicates:
            # Intented usage is for all events to use this. But it doesn't work for some, for now
            if event['name'] == events[idx - 1]['name']:
                duplicates.append(events[idx - 1])

        else:
            # If an event has the same name as the previous one, remove it.
            if event['name'] == events[idx - 1]['name']: 
                events.remove(event)
    
    return duplicates if only_duplicates else events

def scrape_bet7(url, number_of_odds, league):
    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        data = response.json()
    else:
        print(f"Error: {response.status_code}")

    # Only first 2 bets of each match have False, others True. Can also be used 'typeID'==1 or 3
    filtered_events = [(idx, event) for idx, event in enumerate(data['odds']) if event['typeId'] in [1, 2, 3]]
    
    # Desired odds have 1, 2 or 3 as typeId. But also each event has extra 1 and 3. We filter second 1 and 3.
    if number_of_odds == 3:
        first_odds = [event for idx, event in filtered_events if event['typeId'] == 1]
        first_odds = remove_duplicates(first_odds)
        second_odds = [event for idx, event in filtered_events if event['typeId'] == 2]  
        third_odds = [event for idx, event in filtered_events if event['typeId'] == 3]
        third_odds = remove_duplicates(third_odds)
        events = [i for i in zip(first_odds, second_odds, third_odds)]

    elif number_of_odds == 2:
        first_odds = [event for idx, event in filtered_events if event['typeId'] == 1]
        if league == 'Smiginis':
            first_odds = remove_duplicates(first_odds, True) # only remove duplicated terms
        else:    
            first_odds = remove_duplicates(first_odds)
        second_odds = [event for idx, event in filtered_events if event['typeId'] == 3]  
        second_odds = remove_duplicates(second_odds) 
        events = [i for i in zip(first_odds, second_odds)]

    matches_bet7 = []
    # Extract only price and name from each event
    for group in events:
        if len(group) == 2:
            # Extracting the first and second team's names and prices
            team1, team2 = unidecode(group[0]['name']), unidecode(group[1]['name'])
            team1 = team1.replace(',', '')
            team2 = team2.replace(',', '')
            odd1, odd2 = group[0]['price'], group[1]['price']
            
            # Round odds
            odd1 = round(odd1, 2)
            odd2 = round(odd2, 2)
            
            # Append the tuple to the matches list
            matches_bet7.append(((team1, team2), (odd1, odd2)))

        elif len(group) == 3:
            # Extracting the first and second team's names and prices
            team1 = unidecode(group[0]['name']) # Convert lt characters to latin, if exists
            middle = group[1]['name']
            team2 = unidecode(group[2]['name'])
            odd1, odd2, odd3 = group[0]['price'], group[1]['price'], group[2]['price']
            
            if middle == "Lygiosios":
                middle = "Draw"
            else: # something went wrong
                continue

            # Round odds
            odd1 = round(odd1, 2)
            odd2 = round(odd2, 2)
            odd3 = round(odd3, 2)
            
            # Append the tuple to the matches list
            matches_bet7.append(((team1, middle, team2), (odd1, odd2, odd3)))

    return matches_bet7


all_matches = []

for league in leagues.keys():
    number_of_odds = leagues[league][0]
    url = leagues[league][1]
    matches = scrape_bet7(url, number_of_odds, league)
    all_matches.extend([(teams, odds, league) for teams, odds in matches])

# Create a single DataFrame
df_7bet = pd.DataFrame(all_matches, columns=["Teams", "Odds", "League"])

# df_7bet

In [9]:
# Set options
options = Options()
options.add_argument("--headless")  # Run in headless mode
options.add_argument("--disable-gpu")  # Disable GPU acceleration
options.add_argument("--no-sandbox")  # Disables sandbox for the browser
options.add_argument("--disable-images")  # Disable images

# Set urls. Integers represent number of bets to include
url_list_betsafe = [(2, "https://www.betsafe.lt/en/betting/basketball/north-america/nba"), # Basketball 
                    (2, "https://www.betsafe.lt/en/betting/basketball/europe/euroleague"), # Basketball
                    (3, "https://www.betsafe.lt/en/betting/football/europe/uefa-europa-league"), # Football
                    (3, "https://www.betsafe.lt/en/betting/football/europe/uefa-europa-conference-league"), # Football
                    (2, "https://www.betsafe.lt/en/betting/american-football/north-america/nfl"), # NFL (American Footbal)
                    (3, "https://www.betsafe.lt/en/betting/ice-hockey"), # NHL (Ice hockey)
                    (2, "https://www.betsafe.lt/en/betting/darts/world/pdc-world-championships") # Darts
                    ]

league_names = ['NBA', 'Eurolyga', 'UEFA', 'UEFA Konf.', "NFL", "NHL", "Smiginis"]

def scrape_betsafe(url, number_of_odds):
    # Start WebDriver
    driver = webdriver.Firefox(options=options)
    driver.get(url)

    try:
        # List to store all matches
        matches_betsafe = []
        
        # Wait for the events to load
        events = WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "wpt-table__row"))
        )
        
        # Loop through events
        for event in events:
            try:
                # Extract match date and time
                date_elements = event.find_elements(By.CLASS_NAME, "wpt-time")
                if date_elements:
                    current_year = datetime.now().year
                    date_text = date_elements[0].text  # e.g., 04 Dec\n20:00
                    date_text = date_text.replace('\n', f'{current_year} ')

                    # Convert to datetime object
                    date = datetime.strptime(date_text, "%d %b %H:%M")
                    year = current_year # Later implement logic for next year
                    date = date.replace(year=year)

                    # Format the date as year-month-day
                    date = date.strftime("%Y-%m-%d %H:%M")  # Example: "2024-12-04 20:00"

                # Extract teams and odds
                teams = event.find_elements(By.CLASS_NAME, "wpt-teams__team")
                odds_elements = event.find_elements(By.CLASS_NAME, "wpt-odd-changer")
                odds = [odd.text for odd in odds_elements if odd.text]  # Extract non-empty text

                if not odds or len(teams) != 2:
                    continue

                if len(odds) >= 2 and number_of_odds == 2: # There are 4 extra bets that are excluded
                    team1 = teams[0].find_element(By.TAG_NAME, "a").text
                    team2 = teams[1].find_element(By.TAG_NAME, "a").text
                    team1, team2 = unidecode(team1), unidecode(team2)
                    odd1 = float(odds[0])
                    odd2 = float(odds[1])
                    matches_betsafe.append((date, (team1, team2), (odd1, odd2)))

                if len(odds) >= 3 and number_of_odds == 3:
                    team1 = teams[0].find_element(By.TAG_NAME, "a").text
                    team2 = teams[1].find_element(By.TAG_NAME, "a").text
                    team1, team2 = unidecode(team1), unidecode(team2)
                    odd1 = float(odds[0])
                    odd2 = float(odds[1])
                    odd3 = float(odds[2])
                    matches_betsafe.append((date, (team1, "Draw" ,team2), (odd1, odd2, odd3)))
            except:
                continue

            
    finally:
        # Close the browser
        driver.quit()
        return matches_betsafe

all_matches = []

for i, league_name in enumerate(league_names):
    number_of_odds = url_list_betsafe[i][0]
    matches = scrape_betsafe(url_list_betsafe[i][1], number_of_odds)
    all_matches.extend([(date, teams, odds, league_name) for date, teams, odds in matches])

# Create a single DataFrame
df_betsafe = pd.DataFrame(all_matches, columns=["Date", "Teams", "Odds", "League"])
# df_betsafe

## Merge dataframes in one

In [6]:
# Handle tuples to match names from different sites
def teams_match(teams1, teams2):
    # Remove words like "FC" since it's very common and code would still match "FC A" with "FC B"
    teams1 = [team.lower().replace("fc", "") for team in teams1]
    teams2 = [team.lower().replace("fc", "") for team in teams2]
    
    return all(
        any(word1 in team2 for team2 in teams2 for word1 in team1.split()) 
        for team1 in teams1
    ) or all(
        any(word2 in team1 for team1 in teams1 for word2 in team2.split()) 
        for team2 in teams2
    )


# Define the function to merge odds
def merge_odds(df_topsport, df_betsafe, df_7bet):
    merged_matches = []

    matched_betsafe_indices = set()
    matched_7bet_indices = set()

    # Iterate through rows in df_topsport
    for _, topsport_row in df_topsport.iterrows():
        top_date = topsport_row['Date']
        top_teams = topsport_row['Teams']
        top_league = topsport_row['League']
        matched_betsafe_odds = None
        matched_7bet_odds = None

        # Match with betsafe
        for betsafe_index, betsafe_row in df_betsafe.iterrows():
            if betsafe_index in matched_betsafe_indices:
                continue
            if (
                betsafe_row['League'] == top_league
                and teams_match(top_teams, betsafe_row['Teams'])
            ):
                matched_betsafe_odds = betsafe_row['Odds']
                matched_betsafe_indices.add(betsafe_index)
                break

        # Match with 7bet
        for _7bet_index, _7bet_row in df_7bet.iterrows():
            if _7bet_index in matched_7bet_indices:
                continue
            if (
                _7bet_row['League'] == top_league
                and teams_match(top_teams, _7bet_row['Teams'])
            ):
                matched_7bet_odds = _7bet_row['Odds']
                matched_7bet_indices.add(_7bet_index)
                break

        # Append the result
        merged_matches.append(
            {
                'Date': top_date,
                'Match': top_teams,
                'League': top_league,
                'Topsport': topsport_row['Odds'],
                'Betsafe': matched_betsafe_odds,
                '7bet': matched_7bet_odds,
            }
        )

    # Iterate through unmatched rows of df_betsafe
    df_betsafe_unmatched = df_betsafe.drop(matched_betsafe_indices)
    for _, betsafe_row in df_betsafe_unmatched.iterrows():
        bet_date = betsafe_row['Date']
        bet_teams = betsafe_row['Teams']
        bet_league = betsafe_row['League']
        matched_7bet_odds = None

        # Match with 7bet
        for _7bet_index, _7bet_row in df_7bet.iterrows():
            if _7bet_index in matched_7bet_indices:
                continue
            if (
                _7bet_row['League'] == bet_league
                and teams_match(bet_teams, _7bet_row['Teams'])
            ):
                matched_7bet_odds = _7bet_row['Odds']
                matched_7bet_indices.add(_7bet_index)
                break

        # Append the result
        merged_matches.append(
            {
                'Date': bet_date,
                'Match': bet_teams,
                'League': bet_league,
                'Topsport': None,
                'Betsafe': betsafe_row['Odds'],
                '7bet': matched_7bet_odds,
            }
        )
    # Convert merged_matches to DataFrame
    return pd.DataFrame(merged_matches)

df = merge_odds(df_topsport, df_betsafe, df_7bet)
# Sort the DataFrame by 'League' and then by 'Date' within each league
df = df.sort_values(by=['League', 'Date'], ascending=[True, True]).reset_index(drop=True)
df

,Date,Match,League,Topsport,Betsafe,7bet
0,2025-01-02 20:00,"(Zalgiris, Olympiacos)",Eurolyga,"(1.96, 1.78)","(1.95, 1.85)","(2.0, 1.83)"
1,2025-01-02 21:00,"(Alba Berlin, Monaco)",Eurolyga,"(4.7, 1.16)","(5.1, 1.17)","(5.0, 1.18)"
2,2025-01-02 21:00,"(Asvel Lyon Villeurbanne, Milano Armani Exchange)",Eurolyga,"(1.76, 1.99)","(1.8, 2.0)","(1.8, 2.05)"
3,2025-01-02 21:30,"(Baskonia Vitoria Gasteiz, Crvena Zvezda)",Eurolyga,"(1.76, 1.99)","(1.8, 2.0)","(1.8, 2.05)"
4,2025-01-03 19:45,"(Fenerbahce, Anadolu Efes)",Eurolyga,"(1.59, 2.27)","(1.6, 2.35)","(1.62, 2.35)"
...,...,...,...,...,...,...
126,2025-02-13 22:00,"(Backa Topola, Draw, Jagiellonia Bialystok)",UEFA Konf.,"(3.45, 3.7, 2.13)",None,"(3.25, 3.6, 2.05)"
127,2025-02-13 22:00,"(Celje, Draw, Apoel Nicosia)",UEFA Konf.,"(2.34, 3.4, 3.25)",None,"(2.29, 3.33, 3.0)"
128,2025-02-13 22:00,"(Omonia Nicosia, Draw, Pafos FC)",UEFA Konf.,"(2.43, 3.55, 2.95)",None,"(2.36, 3.4, 2.83)"
129,2025-02-13 22:00,"(FK Borac Banja Luka, Draw, Olimpija Ljubljana)",UEFA Konf.,"(3.0, 3.35, 2.5)",None,"(2.9, 3.25, 2.4)"


## Check arbitrages

In [7]:
def get_stakes(odds: list, profit: float, favor: bool = False, idx: int = None) -> np.ndarray:
    # Given odds, calculate normal or favored bets with desired profit

    odds = np.array(odds)
    arbitrage = np.sum(1/odds)

    if favor: # Calculate favored bets, profit will be (0, 0,...,profit,..., 0) at index idx
        total_stake = (profit / odds[idx]) / (1 - arbitrage)
        stakes = total_stake / odds
        stakes[idx] += profit / odds[idx]

    else: # Calculate normal bets, profit will be (profit, profit, ..., profit)
        total_stake = (profit * arbitrage) / (1 - arbitrage) # changed
        stakes = (total_stake + profit) / odds
    
    return stakes

def round_stakes(odds: list, max_stake: int = 100) -> dict:
    # Given odds, calculate normal or favored bets with rounded numbers

    good_stakes = dict() # Final dict of good stakes
    unique_proportions = set() # Store unique proportions. Multiples of it will give the same profit
    for i in range(200):
        profit = 0.1 + i / 10 # Desired profit from that bet
        stakes_favored = [] # Favored bets
        stakes_normal = [] # Normal bets (equal payouts)
        for idx, _ in enumerate(odds):
            stakes_favored.append(get_stakes(odds, profit, favor= True, idx= idx)) # Get favored bets
            stakes_normal.append(get_stakes(odds, profit)) # Get normal bets 

        # Check if results are close to integers and not already tracked
        for stakes in [stakes_favored, stakes_normal]:
            for idx, stake in enumerate(stakes):
                if np.sum(stake) > max_stake: # Skip if stake is bigger than max_stake
                    continue

                proportion = tuple(np.round(stake / stake.sum(), 3)) # Normalise stake to 3 decimal places
                if np.isclose(stake, np.round(stake), atol=0.0001).all() and proportion not in unique_proportions:
                    payoff = odds * stake - np.sum(stake)
                    # Round payoff and replace -0.0 and 0.0 with 0 (-0.0 is caused from rounding)
                    round_payoff = round_payoff = tuple(0 if abs(x) == 0.0 else x for x in map(float, np.round(payoff, 2)))
                    round_stake = tuple(map(int, np.round(stake)))
                    good_stakes[round_stake] =  round_payoff
                    unique_proportions.add(proportion) # Add successfull proportion to set

    return good_stakes

    
arbitrage_list = []

def display_result(match, odds, bookmaker_odds):
    # For displaying bookmakers, define dictionary
    bookmakers = {0: 'Topsport', 1: 'Betsafe', 2: '7Bet'}

    possible_stakes_dict = round_stakes(odds)
    stakes = possible_stakes_dict.keys()
    profits = possible_stakes_dict.values()

    result_str = f'Match: {' - '.join(match)}.\n'

    # Find which bookmakers have the odds
    selected_bookmakers = []
    for idx, odd in enumerate(odds):
        for bm_idx, bm_tuple in enumerate(bookmaker_odds):
            if bm_tuple: # Skip empty (None) tuples
                if odd == bm_tuple[idx]:  # Check if the odd matches the coordinate in tuple
                    selected_bookmakers.append(bookmakers[bm_idx])
                    break
    
    # Display the bets with bookmakers
    for stake, profit in zip(stakes, profits):
        result_str += (
            f'Bet {stake}, odds {[float(odd) for odd in odds]} in ({'-'.join(selected_bookmakers)}). '
            f'Profit: {profit} ,\n'
        )

    arbitrage_list.append(result_str)
    return

# Function to check for arbitrage opportunities
def check_arbitrage(row):
    # Extract odds for each bookmaker
    odds_top = row['Topsport']
    odds_betsafe = row['Betsafe']
    odds_7bet = row['7bet']

    # Create a list of tuples (bookmaker, odds)
    odds_list = []
    if odds_top is not None:
        odds_list.append((odds_top))
    if odds_betsafe is not None:
        odds_list.append((odds_betsafe))
    if odds_7bet is not None:
        odds_list.append((odds_7bet))

    odds_array = np.array(odds_list) 
    odds_array_t = np.transpose(odds_array)

    min_sum = 2  # Any absurd starting value will work. At least should be 1

    # Generate combinations where each bookmaker provides one odds value
    for combination in it.product(*odds_array_t):
        # Check if the total sum indicates an arbitrage opportunity
        total_sum = round(sum(1 / odd for odd in combination), 5)
        min_sum = min(min_sum, total_sum)
        if total_sum < 1: # If the total sum indicates an arbitrage opportunity (should be less than 1)
            bookmaker_odds = row['Topsport'], row['Betsafe'], row['7bet']
            display_result(row['Match'], combination, bookmaker_odds)

    # Return the results and minimum arbitrage sum, or False if no arbitrage is found
    return min_sum

df['best arbitrage'] = df.apply(check_arbitrage, axis=1, result_type='expand')

display(df[df['best arbitrage'] <  1])

for arbitrage in arbitrage_list[:5]:
    print(arbitrage)

,Date,Match,League,Topsport,Betsafe,7bet,best arbitrage
14,2025-01-10 20:00,"(Zalgiris, Real Madrid)",Eurolyga,"(2.2, 1.6)","(1.85, 1.85)",None,0.99509


Match: Zalgiris - Real Madrid.
Bet (5, 6), odds [2.2, 1.85] in (Topsport-Betsafe). Profit: (0, 0.1) ,
Bet (17, 20), odds [2.2, 1.85] in (Topsport-Betsafe). Profit: (0.4, 0) ,
Bet (37, 44), odds [2.2, 1.85] in (Topsport-Betsafe). Profit: (0.4, 0.4) ,



## Implement Database (Optional)

In [26]:
# Extract Team1 and Team2 from the Teams column
df['team1'] = df['Match'].apply(lambda x: x[0])  # First team
df['team2'] = df['Match'].apply(lambda x: x[-1])  # Last team

# Add a 'draw_possible' column for 3 bet matches
df['draw_possible'] = df['Match'].apply(lambda x: len(x) == 3)

import mysql.connector 

# Extract database information. It should be in the format "HOST,USER,PASSWORD"
with open("database_info.txt") as f: 
    text = f.read()
    HOST, USER, PASSWORD = text.split(',')

# Connect to the database
db = mysql.connector.connect(
    host=HOST,
    user=USER,
    password=PASSWORD,
    database = "ArbitrageAnalysis"
)

cursor = db.cursor()

for _, row in df.iterrows():
    # Check if the match already exists in the matches table, add unique matches only
    check_query = """
    SELECT match_id FROM matches 
    WHERE team1 = %s AND team2 = %s AND match_date = %s
    """
    cursor.execute(check_query, (row['team1'], row['team2'], row['Date']))
    result = cursor.fetchone()
    
    if result:  
        match_id = result[0]  # Match already exists
    else:  
        # Insert the match and retrieve its match_id
        insert_match_query = """
        INSERT INTO matches (team1, team2, league, match_date, draw_possible)
        VALUES (%s, %s, %s, %s, %s)
        """
        cursor.execute(insert_match_query, (row['team1'], 
                                            row['team2'], 
                                            row['League'], 
                                            row['Date'], 
                                            row['draw_possible']))
        match_id = cursor.lastrowid  # Retrieve the newly inserted match_id

    # Insert all matches into scraped_data
    insert_scraped_query = """
    INSERT INTO scraped_data (
        match_id, 
        team1, team2, league, 
        best_arbitrage, 
        odd1_topsport, odd_draw_topsport, odd2_topsport, 
        odd1_betsafe, odd_draw_betsafe, odd2_betsafe, 
        odd1_7bet, odd_draw_7bet, odd2_7bet
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    cursor.execute(
        insert_scraped_query, 
        (
            match_id, 
            row['team1'], row['team2'], row['League'],
            row['best arbitrage'], 

            # Topsport
            row['Topsport'][0] if row['Topsport'] else None,
            row['Topsport'][1] if (row['draw_possible'] and row['Topsport']) else None,
            row['Topsport'][-1] if row['Topsport'] else None,

            # Betsafe
            row['Betsafe'][0] if row['Betsafe'] else None,
            row['Betsafe'][1] if (row['draw_possible'] and row['Betsafe']) else None,
            row['Betsafe'][-1] if row['Betsafe'] else None,

            # 7bet
            row['7bet'][0] if row['7bet'] else None,
            row['7bet'][1] if (row['draw_possible'] and row['7bet']) else None,
            row['7bet'][-1] if row['7bet'] else None,
        )
    )

# Commit the changes 
db.commit()

cursor.close()
db.close()

## Send mail (optional)

In [12]:
import smtplib 
from email.message import EmailMessage 

def send_mail():
    # Send mail with the arbitrage opportunities

    # Extract email and password from mail.txt. It should be in the format "email,password"
    with open("mail.txt") as f: 
        text = f.read()
        mail, password = text.split(',')

    FROM = mail
    PASSWORD = password
    SUBJECT = "Arbitrages"
    TO = FROM

    msg = EmailMessage()
    emoji = "\N{smirking face}"  # Unicode for the smirking face
    msg.set_content(f'How about this {emoji}:\n' + '\n'.join(arbitrage_list))
    msg['Subject'] = SUBJECT
    msg['From'] = FROM
    msg['To'] = TO

    # Send the message via an SMTP server
    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as s:
            s.starttls()  # Start TLS encryption
            s.login(FROM, PASSWORD) 
            s.send_message(msg)
        print("Email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")

if arbitrage_list:
    send_mail()

Email sent successfully!
